<a href="https://colab.research.google.com/github/sneharc16/DTU-MLR-Assignments-Deep-Learning/blob/main/KNN_Assignment_DTU_MLR_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

CIFAR-10 is a well known dataset composed of 60,000 colored 32x32 images. kNN classification is an algorithm to classify inputs by comparing their similarities to a training set accompanied with labels.

# **What is kNN / How does kNN work**



The goal is classification. We have a dataset, images in our case, and unfortunately a portion of the images are unlabeled - they do not have an accompanying label which classifies them. kNN is a method to "intelligibly" label the unclassified images by computing similarity with the labeled images. Euclidean distance is a popular metric to quantify the similarities, however there are a handful of other measures that have their own idiosyncrasies (Manhattan & Minkowsi). In the case of images, kNN computes the Euclidean distance between an unlabled and labeled image by iterating over their pixel values. The unlabeled image then takes the same majority classification as its k nearest neighbors or k neighbors with the lowest Euclidean distance, where k is a hyperparameter. This process produces a loop where each unlabeled input image is compared to every labeled image in the dataset by their pixel values. Once the first unlabeled image iterates through the entire labeled training set, the algorithm then moves onto the next image in the test set to do it all again.

. In the visualization above, kNN is operating on a colored (RGB) image dataset of 5x5 pixels - so in total 75 pixels per image (3x5x5). To compute the Eucliden distance between one test image and one training image, the squared difference between corresponding pixels in both images are taken and then then summed and finally taken the square root of. To reiterate, we are taking 75 differences, squaring each one, summing all that together, and then finally square rooting everything. And this occurs for each test image against every training image. It's worth mentioning, in the animation I do not show the 5x5 green and blue pixel channels being iterated over like I did for the red channel; instead the green and blue layers are simplified with a single "blink".

As mentioned briefly before, the k in kNN is an integer hyperparameter that says how many neighbors our test image should pay attention to. It tells each test image to find their k nearest neighbors of a particular label, then label them in accordance with those neighbors. If k = 1, then we're asking kNN to classify every test image with it's closest single neighbor. If k = 3, then we're asking kNN to classify a test image with it's 3 closest neighbors (lowest Euclidean distance) of the same class. Some considerations when picking a value for k is to not pick a value that would result in a tie - where the k closest neighbors are an even distribution between different classes. This can generally be avoided by 1) picking odd numbers for k and 2) not picking multiples of the number of classes.

# Loading CIFAR-10
Lets now look at using kNN on CIFAR-10. Our data is going to be stored simply in the four variables: x_train, x_test, y_train, and y_test.


In [ ]:
#import os
import torch
from torchvision.datasets import CIFAR10
from torchvision.transforms import ToTensor

def _extract_tensors(dset, num=None):
  x = torch.tensor(dset.data, dtype=torch.float32).permute(0, 3, 1, 2).div_(255)
  y = torch.tensor(dset.targets, dtype=torch.int64)
  if num is not None:
    if num <= 0 or num > x.shape[0]:
      raise ValueError('Invalid value num=%d; must be in the range [0, %d]' % (num, x.shape[0]))
    x = x[:num].clone()
    y = y[:num].clone()
    return x, y

def cifar10():
      #download = not os.path.isdir('cifar-10-batches-py')
  dset_train = CIFAR10(
    root="data",
    train=True,
    download=True,
    transform=ToTensor())

  dset_test = CIFAR10(
    root="data",
    train=False,
    download=True,
    transform=ToTensor())
  x_train, y_train = _extract_tensors(dset_train, len(dset_train))
  x_test, y_test = _extract_tensors(dset_test, len(dset_test))

  return x_train, y_train, x_test, y_test

In [ ]:

# Redeclaring x_train...y_test with subsample
x_train, y_train, x_test, y_test = cifar10()

print('data shape:', x_train.shape)
print('labels shape:', y_train.shape)
# Training set:
# >>> data shape: torch.Size([500, 3, 32, 32])
# >>> labels shape: torch.Size([500])

print('data shape:', x_test.shape)
print('labels shape:', y_test.shape)
# Testing set:
# >>> data shape: torch.Size([250, 3, 32, 32])
# >>> labels shape: torch.Size([250])

Files already downloaded and verified
Files already downloaded and verified
data shape: torch.Size([50000, 3, 32, 32])
labels shape: torch.Size([50000])
data shape: torch.Size([10000, 3, 32, 32])
labels shape: torch.Size([10000])


# Building kNN
## Finding the Euclidean distance
Everything is set - we've preprocessed and subsampled our dataset. I'm going to show a couple ways to find the Euclidean distance between testing and training images. The first is with using nested loops to populate our output matrix dists, where each loop will iterate over an axis. Although this method is not very efficient because it does not make use of broadcasting, I think it's helpful from a pedagogical standpoint. A more efficient representation, compute_distances_no_loops is shown immediately below. It is worth mentioning that PyTorch does have functions designed to do this already such as: torch.cdist, but I'm also going to refrain from using them. Broadcasting is a term that enables arithmetic for tensors of different dimensionality, read more here.

In [ ]:
def compute_distances_two_loops(x_train, x_test):
      """
      Inputs:
      x_train: shape (num_train, C, H, W) tensor.
      x_test: shape (num_test, C, H, W) tensor.

      Returns:
      dists: shape (num_train, num_test) tensor where dists[j, i] is the
         Euclidean distance between the ith training image and the jth test
         image.
      """

      # Get the number of training and testing images
      num_train = x_train.shape[0]
      num_test = x_test.shape[0]


      # dists will be the tensor housing all distance measurements between testing and training
      dists = x_train.new_zeros(num_train, num_test)

      # Flatten tensors
      train = x_train.flatten(1)
      test = x_test.flatten(1)

      # Find Euclidean distance using loops
      for i in range(num_test):
         for j in range(num_train):
            dists[j, i] = torch.sqrt(torch.sum(torch.square(train[j] - test[i])))


      return dists

As stated before, CIFAR-10 images are 3x32x32. The entire training dataset can be represented with a tensor of shape [50000, 3, 32, 32].  We flatten both the the x_train and x_test tensors to transform our [50000, 3, 32, 32] and [10000, 3, 32, 32] training image tensor into [50000, 3072] and [10000, 3072] tensors respectively. Now you can think of our reshaped tensors as matrices where each row houses every single pixel value (3x32x32) of an image. The purpose of flattening can be thought of as the final preprocessing step before we compute the Euclidean distance using two loops.

The first for loop iterates over every test image. The second iterates over every training image. Like discussed before, it computes the Euclidean distance between the jth training image and ith testing image two and populates the dists tensor in its respective position.

Here is a more efficient variation of finding the Euclidean distance that has no loops and instead makes use of broadcasting:

In [ ]:
def compute_distances_no_loops(x_train, x_test):
      """
      Inputs:
      x_train: shape (num_train, C, H, W) tensor.
      x_test: shape (num_test, C, H, W) tensor.

      Returns:
      dists: shape (num_train, num_test) tensor where dists[i, j] is the
         Euclidean distance between the ith training image and the jth test
         image.
      """
      # Get number of training and testing images
      num_train = x_train.shape[0]
      num_test = x_test.shape[0]

      # Create return tensor with desired dimensions
      dists = x_train.new_zeros(num_train, num_test) # (50000, 10000)

      # Flattening tensors
      train = x_train.flatten(1) # (50000, 3072)
      test = x_test.flatten(1) # (10000, 3072)

      # Find Euclidean distance
      # Squaring elements
      train_sq = torch.square(train)
      test_sq = torch.square(test)

      # Summing row elements
      train_sum_sq = torch.sum(train_sq, 1) # (50000)
      test_sum_sq = torch.sum(test_sq, 1) # (10000)

      # Matrix multiplying train tensor with the transpose of test tensor
      mul = torch.matmul(train, test.transpose(0, 1)) # (50000, 10000)

      # Reshape enables proper broadcasting.
      # train_sum_sq = [500, 1] shape tensor and test_sum_sq = [1, 10000] shape tensor.
      # This enables broadcasting to match desired dimensions of dists
      dists = torch.sqrt(train_sum_sq.reshape(-1, 1) + test_sum_sq.reshape(1, -1) - 2*mul)

      return dists

This procedure makes use of expanding the square in the Euclidean distance.
On this no-loop variant of computing the Euclidean, we are evaluating all arithmetic independently and then compiling everything together so that it represents the right hand side of the formula above. Instead of going through element by element, after we preprocess the inputs, we square the train and test tensors immediately then take the sum of their rows. Then, xi ,yi
 is evaluated through the matrix multiplication of the train [500, 3072] and the transpose of the test [3072, 250] tensors. Finally, we have all terms to produce the right hand of the equality above, allowing us to wrap everything in a square root and store it in dists. Within each column of dists is the Euclidean distance of every training image with respect to a test image. For comparison, the two loop version takes (for me) 7.27 seconds. The no loop version takes 0.02 seconds. The no loop version is 455.7x faster than the two loop. This probably provides better intuition behind how powerful broadcasting (and matmul) can be.

## Classifying our test images
Using the compute_distances_no_loops function, we can now build a classifier. I will be focusing on the predict method. check_accuracy is simply to determine how well our classifier performs.

In [ ]:
class KnnClassifier:
      def __init__(self, x_train, y_train):
         """
         x_train: shape (num_train, C, H, W) tensor where num_train is batch size,
            C is channel size, H is height, and W is width.
         y_train: shape (num_train) tensor where num_train is batch size providing labels
         """

         self.x_train = x_train
         self.y_train = y_train

      def predict(self, x_test, k=1):
         """
         x_test: shape (num_test, C, H, W) tensor where num_test is batch size,
            C is channel size, H is height, and W is width.
         k: The number of neighbors to use for prediction
         """

         # Init output shape
         y_test_pred = torch.zeros(x_test.shape[0], dtype=torch.int64)

         # Find & store Euclidean distance between test & train
         dists = compute_distances_no_loops(self.x_train, x_test)

         # Index over test images
         for i in range(dists.shape[1]):
            # Find index of k lowest values
            x = torch.topk(dists[:,i], k, largest=False).indices

            # Index the labels according to x
            k_lowest_labels = self.y_train[x]

            # y_test_pred[i] = the most frequent occuring index
            y_test_pred[i] = torch.argmax(torch.bincount(k_lowest_labels))

         return y_test_pred

      def check_accuracy(self, x_test, y_test, k=1, quiet=False):
         """
         x_test: shape (num_test, C, H, W) tensor where num_test is batch size,
            C is channel size, H is height, and W is width.
         y_test: shape (num_test) tensor where num_test is batch size providing labels
         k: The number of neighbors to use for prediction
         quiet: If True, don't print a message.

         Returns:
         accuracy: Accuracy of this classifier on the test data, as a percent.
            Python float in the range [0, 100]
         """

         y_test_pred = self.predict(x_test, k=k)
         num_samples = x_test.shape[0]
         num_correct = (y_test == y_test_pred).sum().item()
         accuracy = 100.0 * num_correct / num_samples
         msg = (f'Got {num_correct} / {num_samples} correct; '
               f'accuracy is {accuracy:.2f}%')
         if not quiet:
            print(msg)
         return accuracy

We've finished implementing kNN and can begin testing the algorithm on larger portions of the dataset to see how well it performs. With k set to 5, 5000 training images, and 500 testing images, our kNN results in a 27.8% accuracy for proper classification on a portion of the CIFAR-10 dataset. It's certainly no convolutional neural network, however it shows how far computer vision has come (kNN was developed in 1951).



In [ ]:
#import KnnClassifier

torch.manual_seed(0)
x_train, y_train, x_test, y_test = cifar10()

classifier = KnnClassifier(x_train, y_train)
classifier.check_accuracy(x_test, y_test, k=2)


Files already downloaded and verified
Files already downloaded and verified
Got 3115 / 10000 correct; accuracy is 31.15%


31.15

# Cross Validation
A problem with the current kNN is that after we've successfully automated a process of "intelligebly" classifying images, we are still left with "guessing" the best k for the algorithm. As our algorithm currently exists, we have to manually tune the hyperparameter k to some integer value, which raises the question - is that the best value for k? Cross validation is a procedure to automate selecting an optimal value for k. The problem with this is that to evaluate the efficacy of some value k, we currently use the test set. If we want to evaluate numerous values for k it breaks the convention of segregating the test set until final testing. Finding some optimal k through using the same test set on each evaluation, would have our kNN algorithm fall victim to overfitting. Our model would be too well trained to the test set, and because of this, may not generalize well to new and unseen data. Overfitting is a big issue in training. The inverse of overfitting is underfitting, where a model does not yield high enough or desired efficacy on training data.

Cross validation further segregates our training set into "chunks". For our subsample of 5,000 test images and labels, we can create 5 tensors of shape [1000, 3072]. One of those 5 chunks becomes what is called the validation set to evaluate an optimal k. The validation set does the job of the previously defined test set. By partioning our data, we circumvent the issue of overfitting - our test set is untouched until the final...test.

In [ ]:
def knn_cross_validate(x_train, y_train, num_folds=5, k_choices=None):
      """
      Inputs:
      x_train: Tensor of shape (num_train, C, H, W) giving all training data
      y_train: int64 tensor of shape (num_train,) giving labels for training data
      num_folds: Integer giving the number of folds to use
      k_choices: List of integers giving the values of k to try

      Returns:
      k_to_accuracies: Dictionary mapping values of k to lists, where
         k_to_accuracies[k][i] is the accuracy on the ith fold of a KnnClassifier
         that uses k nearest neighbors.
      """
      # Create a list of k's for testing
      if k_choices is None:
         k_choices = [1, 3, 5, 8, 10, 12, 15, 20, 50, 100]

      # Create empty lists to house the chunks for cross validation
      x_train_folds = []
      y_train_folds = []

      # Flatten x_train from [50000, 3, 32, 32] to [50000, 3072]
      x_train_flat = x_train.view(x_train.shape[0], -1)

      # Partition our training set to 5 tensors of training images of shape [10000, 3072] and 5 labels of shape [10000]
      x_train_folds = torch.chunk(x_train_flat, num_folds, dim=0)
      y_train_folds = torch.chunk(y_train, num_folds, dim=0)

      # Create object to house the combination of accuracies for different values k for different validation sets
      k_to_accuracies = {}

      # Iterate through every combination of k_choices and possible validation sets
      for k in k_choices:
         for folds in range(num_folds):
            # Setting validation sets
            x_valid = x_train_folds[folds]
            y_valid = y_train_folds[folds]

            # Setting new training sets
            x_traink = torch.cat(x_train_folds[:folds] + x_train_folds[folds + 1:])
            y_traink = torch.cat(y_train_folds[:folds] + y_train_folds[folds + 1:])

            # Call our kNN with the newly defined sets
            knn = KnnClassifier(x_traink, y_traink)

            # Check accuracy
            accuracy = knn.check_accuracy(x_valid, y_valid, k=k)

            # Append a list of accuracies for different values k to k_to_accuracies
            k_to_accuracies.setdefault(k, []).append(accuracy)

            return k_to_accuracies

In [ ]:
def knn_get_best_k(k_to_accuracies):
      """
      Inputs:
      - k_to_accuracies: Dictionary mapping values of k to lists, where
      k_to_accuracies[k][i] is the accuracy on the ith fold of a KnnClassifier
      that uses k nearest neighbors.

      Returns:
      - best_k: best (and smallest if there is a conflict) k value based on
      the k_to_accuracies info
      """
      # Create best_k variable to return optimal k
      best_k = 0

      # Get keys and values from k_to_accuracies object
      keys = [k for k in k_to_accuracies.keys()]
      # >>> keys = [1, 3, 5, 8, 10, 12, 15, 20, 50, 100]
      values = [v for v in k_to_accuracies.values()]
      # >>> values = [[26.3, 25.7, 26.4, 27.8, 26.6], ..., [25.6, 27.0, 26.3, 25.6, 26.3]]

      # Get largest average of all the values
      max_avg = torch.argmax(torch.mean(torch.tensor(values), dim=1))
      # Get corresponding k for max_avg
      best_k = keys[max_avg]

      return best_k

      # >>> Best k is  10
      # >>> Got 141 / 500 correct; accuracy is 28.20%
      # >>> 28.2

In [ ]:
#from knn import KnnClassifier

torch.manual_seed(0)
x_train_all, y_train_all, x_test_all, y_test_all = cifar10()
k_to_accuracies = knn_cross_validate(x_train_all, y_train_all, num_folds=5, k_choices=None)
best_k = knn_get_best_k(k_to_accuracies)
classifier = KnnClassifier(x_train_all, y_train_all)
classifier.check_accuracy(x_test_all, y_test_all, k=best_k)

   # >>> Got 3399 / 10000 correct; accuracy is 33.99%
   # >>> 33.99

Files already downloaded and verified
Files already downloaded and verified
Got 3358 / 10000 correct; accuracy is 33.58%
